In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [3]:
file_path = "20240310_final_dataset_b2b.csv"
data = pd.read_csv(file_path)

In [4]:
data.head()

,date_order,date_invoice,product_id,client_id,sales_net,quantity,branch_id,product_price,order_channel_at the store,order_channel_by phone,order_channel_during the visit of a sales rep,order_channel_online,order_channel_other,time_diff,mean_purchase_frequency_days,cluster
0,2019-08-13,2019-08-13,1650826,6,60.858000,3,8470,20.286000,False,True,False,False,False,NaN,55,2
1,2019-07-31,2019-07-31,942611,6,41.538000,3,8470,13.846000,True,False,False,False,False,13.0,55,2
2,2019-07-31,2019-07-31,113967,6,63.673200,3,8470,21.224400,True,False,False,False,False,0.0,55,2
3,2019-07-31,2019-07-31,1522271,6,370.653924,401,8470,0.924324,True,False,False,False,False,0.0,55,2
4,2019-07-25,2019-07-25,1650826,6,60.858000,3,8470,20.286000,False,True,False,False,False,6.0,55,2


In [5]:
columns_to_delete = ["order_channel_at the store",
                     "order_channel_by phone",
                     "order_channel_during the visit of a sales rep",
                     "order_channel_online",
                     "branch_id",
                     "order_channel_other",
                     "cluster"]

# Drop the specified columns
data.drop(columns=columns_to_delete, inplace=True)

In [6]:
product_counts = data['product_id'].value_counts()

# Select the top 100 most bought products
top_100_products = product_counts.head(100).index.tolist()

In [7]:
data_grouped = data.groupby(['date_order', 'client_id'])['product_id'].agg(basket=lambda x: x.tolist()).reset_index()

In [8]:
data_grouped.head()

,date_order,client_id,basket
0,2017-09-22,403403,"[56891, 2995430, 532445]"
1,2017-09-22,585799,"[183015, 452441, 482165, 2634108]"
2,2017-09-22,591886,"[1856047, 1856047]"
3,2017-09-22,688261,"[1800882, 2821712, 43268, 2382607, 2636458, 68..."
4,2017-09-22,791423,"[1617001, 529686, 154693]"


In [9]:
data_filtered_top100 = data_grouped[data_grouped['basket'].apply(lambda basket: any(item in top_100_products for item in basket))]

In [46]:
data_filtered_top100_subset = data_filtered_top100.sample(frac=0.25, random_state=42)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
transactions = data_filtered_top100_subset['basket'].tolist()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
# Assuming 'support' column needs to be converted to numeric
rules['support'] = pd.to_numeric(rules['support'])

# Now you can use nlargest
top_rules = rules.nlargest(10, 'support')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
top_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6,(738406),(1390401),0.064736,0.099814,0.048833,0.754336,7.557431,0.042371,3.664301,0.927738
49,"(1390401, 2734140)",(738406),0.028003,0.064736,0.021911,0.782462,12.086927,0.020098,4.299313,0.943692
50,"(2734140, 738406)",(1390401),0.026265,0.099814,0.021911,0.834232,8.357882,0.019289,5.430407,0.904099
8,(1240112),(932375),0.023964,0.026246,0.020838,0.869548,33.130648,0.020209,7.464466,0.993627
9,(932375),(1240112),0.026246,0.023964,0.020838,0.793931,33.130648,0.020209,4.736447,0.995956
13,(1862474),(1180925),0.022037,0.023452,0.020677,0.938286,40.009564,0.020160,15.823702,0.996976
14,(1180925),(1862474),0.023452,0.022037,0.020677,0.881690,40.009564,0.020160,8.266081,0.998420
11,(1438570),(1180925),0.023048,0.023452,0.019786,0.858489,36.606954,0.019246,6.900880,0.995630
12,(1180925),(1438570),0.023452,0.023048,0.019786,0.843700,36.606954,0.019246,6.250482,0.996041
18,(1862474),(1438570),0.022037,0.023048,0.018816,0.853857,37.047673,0.018309,6.684914,0.994933


In [53]:
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.013780,(91564)
1,0.023358,(91695)
2,0.023173,(114938)
3,0.015128,(120144)
4,0.019308,(123753)
...,...,...
219,0.010319,"(1862474, 571498, 1240112, 932375, 1180925)"
220,0.010477,"(571498, 1862474, 1438570, 932375, 1180925)"
221,0.010060,"(1862474, 1438570, 571498, 1240112, 932375)"
222,0.010172,"(571498, 1862474, 1438570, 1240112, 1180925)"


In [54]:
df_encoded.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952944 entries, 0 to 952943
Columns: 185726 entries, 15 to 3238833
dtypes: bool(185726)
memory usage: 164.8 GB
